# Scraping Markets calendar & Trending Tickers from Yahoo Finance

TODO  (Intro): 
- Introduction about web scraping
- Introduction about and the problem statement
- Mention the tools you're using (Python, requests, Beautiful Soup, Pandas)


Here are the steps we'll follow:

Phase 1
- We're going to scrape following 2 websites using 2 different techniques

a. https://finance.yahoo.com/calendar
    - this page is showing date wise market events data in tabular format for Earnings , Stock Splits, Economic Events. our aim is to create script which can be run form single date and single event and grab the data and save it in csv format.
    - if no data then just create tiles with column header , or blank based on parameter.
    
    
b. https://finance.yahoo.com/trending-tickers
 - selenium method and find tag method to scrape this website 
 
Phase 2 

Automate this process to get daily calendar & trending tickers in csv files
- create daily 5 files 
- move the archive files in to archive folder with time stamp 
- delete older files files older than 2 weeks


In [1]:
!pip install jovian --upgrade --quiet

In [2]:
import jovian

In [3]:
# Execute this to save new versions of the notebook
jovian.commit(project="scrap-market-calendar-trending-tickers")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "vinodvidhole/scrap-market-calendar-trending-tickers" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/vinodvidhole/scrap-market-calendar-trending-tickers


'https://jovian.ai/vinodvidhole/scrap-market-calendar-trending-tickers'

In [4]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [5]:
def get_event_page(scraper_url):
    """add something"""
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
                  "(KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
    }
    response = requests.get(scraper_url, headers=headers)
    if not response.ok:
        print('Status code:', response.status_code)
        raise Exception('Failed to fetch web page ' + scraper_url)
    
    # Construct a beautiful soup document
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [6]:
def get_json_dictionary(doc):
    """Add"""
    ## Explain 
    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = doc.find('script', text=pattern).text
    
    ## Explain
    start  = script_data.find('context')-2
    json_dictionary  = script_data[start:-12]
    
    ##explain 
    parsed_dictionary = json.loads(json_dictionary)
    
    return parsed_dictionary, json_dictionary    

In [7]:
# this function is useful during analysis 
def create_json_file(file_name,json_dictionary):
    """Add"""
    with open(file_name+'.json', 'w') as file:
        file.write(json_dictionary)

In [8]:
def get_columns_and_total_rows(parsed_dictionary):
    #explain 
    total_rows = parsed_dictionary['context']['dispatcher']['stores']['ScreenerResultsStore']['results']['total']
    column_dictionary = parsed_dictionary['context']['dispatcher']['stores']['ScreenerResultsStore']['results']['columns']
    
    return total_rows, column_dictionary

In [9]:
def get_page_rows(parsed_dictionary):
    data_dictionary = parsed_dictionary['context']['dispatcher']['stores']['ScreenerResultsStore']['results']['rows']
    
    return len(data_dictionary), data_dictionary

In [10]:
def scrape_all_pages(event_type, date):
    base_url = 'https://finance.yahoo.com/calendar/{}?day={}&offset={}&size={}'
    page_size = '100' # this indicates max rows per page 
    page_number = 1
    pagewise_rows = 0
    final_data_dictionary = []
    
    while page_number > 0:
        #Explain : starting page 0, next page must be multiple of page_size i.e. 100
        print("Pricessing page # {}".format(page_number))
        page_url = str((page_number - 1 ) * int(page_size))
        scrape_url = base_url.format(event_type, date, page_url, page_size)
        print("Scrape url for page {} is {}".format(page_number,scrape_url))
        page_doc = get_event_page(scrape_url)
        parse_dict, json_dict = get_json_dictionary(page_doc)
        total_row, column_dict = get_columns_and_total_rows(parse_dict)        
        page_rows , data_dict = get_page_rows(parse_dict)
        print("total rows for page {} : {}".format(page_number,page_rows))
        pagewise_rows += page_rows
        final_data_dictionary += data_dict
        if pagewise_rows >= total_row:
            page_number = 0
            return final_data_dictionary, column_dict
        page_number += 1

In [11]:
def get_dataframe_layout(filter_lists, column_dictionary):
    """Add"""
    filter_columns = []
    filter_columns_header = []
    for i in filter_lists:
        filter_columns.append(column_dictionary[i]['data'])
        filter_columns_header.append(column_dictionary[i]['content'])
    return filter_columns, filter_columns_header  

In [12]:
def data_processing(df):
    #data processing
    if event_type == 'earnings':
        df['startdatetimetype'].replace(decode_dictionary, inplace=True)
    elif event_type == 'splits':
        df['ratio'] = df['old_share_worth'].astype(str) + '.00 - ' + df['share_worth'].astype(str)+'.00'
        df['optionable'].replace(decode_dictionary, inplace=True)
        df['startdatetime'] = pd.to_datetime(df['startdatetime']).dt.strftime('%b %d, %Y')
    elif event_type == 'economic':
        df['startdatetime'] = pd.to_datetime(df['startdatetime']).dt.strftime('%I:%M%p')
    return df 

In [13]:
def create_csv(data_dictionary, filter_columns, filter_columns_header):
    """"""
    file_name = event_type+'_'+date_param
    if event_type == 'splits':
        filter_columns = csv_columnm[:-2] + ['ratio']
        filter_columns_header = csv_header[:-2] + ['Ratio'] 
            
    if len(data_dictionary) > 0:
        scraped_df = pd.DataFrame(data_dictionary)
        processed_df = data_processing(scraped_df)
        processed_df.to_csv(file_name+'.csv',columns=filter_columns,header=filter_columns_header,index=False)
    else:
        processed_df=pd.DataFrame(columns=filter_columns_header)
        processed_df.to_csv(file_name+'.csv',index=False)

In [14]:
def set_global_variables(event_type):
    #global variables 
    decode_dict  = {'AMC':'After Market Close',
                          'BMO':'Before Market Open',
                          'TAS':'Time As Supplied',
                          'TNS':'Time Not Supplied',
                          False:'N',
                          True:'Y'
                         }
    #explan this 
    if event_type == 'earnings':
        column_filter_id = [0,1,3,4,5,6]
    elif event_type == 'splits':
        column_filter_id = [0,1,2,3,4,5] 
    elif event_type == 'economic':
        column_filter_id = [0,1,2,3,4,5,6,7] 
    return decode_dict, column_filter_id    

In [15]:
## create dictionary to identify webpage and technique to use 
event_type = 'earnings'
event_type = 'splits'
event_type = 'economic'

# create a function to get default date if date is not passed 
date_param = '2022-02-24'


# create function to showcase all techniques
decode_dictionary, column_id = set_global_variables(event_type)
data_dict, column_dict = scrape_all_pages(event_type, date_param)
csv_columnm , csv_header = get_dataframe_layout(column_id, column_dict)
create_csv(data_dict, csv_columnm, csv_header)

Pricessing page # 1
Scrape url for page 1 is https://finance.yahoo.com/calendar/economic?day=2022-02-24&offset=0&size=100
total rows for page 1 : 80


In [ ]:
jovian.commit(project="scrap-market-calendar-trending-tickers",git_commit=True)

<IPython.core.display.Javascript object>

future 
fix timezone in market events 


-do to

-check above notes 

-testing - done normal, zero rows 

-comments 

-print statements & function doc strings  

-code clean up *** is applicable 

-documentation